## Week 4 Python to Golang code coverter
# A race between open source models

In [ ]:
import os
import gradio as gr
import pandas as pd
import traceback
import time
import subprocess
import tempfile
from huggingface_hub import InferenceClient
from openai import OpenAI

HF_TOKEN = os.getenv("HF_TOKEN", "")
OPENAI_KEY = os.getenv("OPENAI_API_KEY", "")

print(f"HF Token: {'✓ Found' if HF_TOKEN else '✗ Not found'}")
print(f"OpenAI Key: {'✓ Found' if OPENAI_KEY else '✗ Not found'}")

In [ ]:
system_info = "Mac M2, ARM64 architecture"
compile_command = "go build -o main main.go && ./main"

system_prompt = """Your task is to convert Python code into high performance Golang code. 
Respond only with golang code. Do not provide any explanation other than occasional comments. 
The Golang response needs to produce an identical output in the fastest possible time."""

def user_prompt_for(python):
    return f"""Port this Python code to Golang with the fastest possible implementation that produces identical output in the least time.

The system information is: {system_info}
Your response will be written to a file called main.go and then compiled and executed.
The compilation command is: {compile_command}

Respond only with golang code.

Python code to port:
```python
{python}
```
"""

def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
SAMPLE_CODES = {
    "Hello World": "print('Hello, World!')",
    "Fibonacci": """def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

result = fibonacci(10)
print(f'Fibonacci(10) = {result}')""",
    "Sum List": """numbers = [1, 2, 3, 4, 5]
total = sum(numbers)
print(f'Sum: {total}')""",
    "Pi Calculation": """import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")"""
}

HF_MODELS = {
    "CodeLlama-7B": "codellama/CodeLlama-7b-Instruct-hf",
    "StarCoder2-7B": "bigcode/starcoder2-7b",
    "DeepSeek-Coder-6.7B": "deepseek-ai/deepseek-coder-6.7b-instruct",
    "Phi-3-Mini": "microsoft/Phi-3-mini-4k-instruct",
    "Mistral-7B": "mistralai/Mistral-7B-Instruct-v0.2"
}

OPENAI_MODELS = {
    "GPT-4": "gpt-4",
    "GPT-4-Turbo": "gpt-4-turbo-preview",
    "GPT-3.5-Turbo": "gpt-3.5-turbo"
}

ALL_MODELS = {**HF_MODELS, **OPENAI_MODELS}

In [ ]:
def run_python_code(python_code):
    """Run Python code and measure execution time"""
    try:
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(python_code)
            temp_file = f.name
        
        start_time = time.time()
        result = subprocess.run(
            ['python3', temp_file],
            capture_output=True,
            text=True,
            timeout=30
        )
        end_time = time.time()
        
        os.unlink(temp_file)
        
        if result.returncode == 0:
            return end_time - start_time, result.stdout, ""
        else:
            return 0, "", result.stderr
            
    except subprocess.TimeoutExpired:
        return 0, "", "Timeout (30s)"
    except Exception as e:
        return 0, "", str(e)

def run_golang_code(golang_code):
    """Compile and run Go code and measure execution time"""
    try:
        with tempfile.TemporaryDirectory() as temp_dir:
            go_file = os.path.join(temp_dir, "main.go")
            with open(go_file, 'w') as f:
                f.write(golang_code)
            
            compile_result = subprocess.run(
                ['go', 'build', '-o', 'main', 'main.go'],
                cwd=temp_dir,
                capture_output=True,
                text=True,
                timeout=30
            )
            
            if compile_result.returncode != 0:
                return 0, "", f"Compile error: {compile_result.stderr}"
            
            start_time = time.time()
            run_result = subprocess.run(
                ['./main'],
                cwd=temp_dir,
                capture_output=True,
                text=True,
                timeout=30
            )
            end_time = time.time()
            
            if run_result.returncode == 0:
                return end_time - start_time, run_result.stdout, ""
            else:
                return 0, "", run_result.stderr
                
    except subprocess.TimeoutExpired:
        return 0, "", "Timeout (30s)"
    except Exception as e:
        return 0, "", str(e)

In [ ]:
def convert_with_openai(python_code, model_name, api_key):
    try:
        client = OpenAI(api_key=api_key)
        messages = messages_for(python_code)
        
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            max_tokens=2000,
            temperature=0.1
        )
        
        golang_code = response.choices[0].message.content.strip()
        return golang_code
        
    except Exception as e:
        raise Exception(f"OpenAI Error: {str(e)}")

def convert_with_huggingface(python_code, model_name, hf_token):
    try:
        client = InferenceClient(token=hf_token)
        messages = messages_for(python_code)
        
        response = client.chat_completion(
            model=model_name,
            messages=messages,
            max_tokens=2000,
            temperature=0.1
        )
        
        golang_code = response.choices[0].message.content.strip()
        return golang_code
        
    except Exception as e:
        raise Exception(f"HuggingFace Error: {str(e)}")

def convert_python_to_golang(python_code, model_name):
    try:
        if not python_code or not python_code.strip():
            return "", "Error: No code", "Empty input"
        
        is_openai = model_name in OPENAI_MODELS
        model_id = ALL_MODELS.get(model_name)
        
        if not model_id:
            return "", f"Error: Model not found", f"Invalid: {model_name}"
        
        if is_openai:
            if not OPENAI_KEY:
                return "", "Error: No OpenAI key", "Missing OPENAI_KEY"
        else:
            if not HF_TOKEN:
                return "", "Error: No HF token", "Missing HF_TOKEN"
        
        max_retries = 2
        for attempt in range(max_retries):
            try:
                if is_openai:
                    golang_code = convert_with_openai(python_code, model_id, OPENAI_KEY)
                else:
                    golang_code = convert_with_huggingface(python_code, model_id, HF_TOKEN)
                
                if "```go" in golang_code:
                    parts = golang_code.split("```go")
                    if len(parts) > 1:
                        golang_code = parts[1].split("```")[0].strip()
                elif "```golang" in golang_code:
                    parts = golang_code.split("```golang")
                    if len(parts) > 1:
                        golang_code = parts[1].split("```")[0].strip()
                
                if golang_code.startswith("```"):
                    golang_code = golang_code[3:].strip()
                if golang_code.endswith("```"):
                    golang_code = golang_code[:-3].strip()
                
                if golang_code and len(golang_code) > 10:
                    provider = "OpenAI" if is_openai else "HuggingFace"
                    return golang_code, f"✓ Converted with {model_name} ({provider})", ""
                else:
                    return "", "Error: Empty response", "No valid code generated"
                
            except Exception as e:
                error_msg = str(e)
                
                if "503" in error_msg or "loading" in error_msg.lower():
                    return "", f"Model loading. Wait 30s", f"503: Initializing"
                
                elif "404" in error_msg:
                    return "", f"Error 404: '{model_name}' not accessible", f"Try another model"
                
                elif "429" in error_msg:
                    return "", "Rate limit. Wait 1 min", "429: Too many requests"
                
                elif "401" in error_msg or "403" in error_msg:
                    return "", "Invalid API key", "Check credentials"
                
                else:
                    if attempt < max_retries - 1:
                        time.sleep(3)
                        continue
                    return "", f"Error: {error_msg[:100]}", traceback.format_exc()
        
        return "", "All retries failed", "Max attempts exceeded"
        
    except Exception as e:
        return "", f"Error: {str(e)[:100]}", traceback.format_exc()

def write_output(go_code):
    try:
        with open("main.go", "w", encoding="utf-8") as f:
            f.write(go_code)
        return "✓ Written to main.go"
    except Exception as e:
        return f"Error writing file: {str(e)}"

In [ ]:
custom_css = """
.gradio-container {
    font-family: 'Inter', sans-serif;
}
.main-header {
    text-align: center;
    padding: 2rem 0;
    background: linear-gradient(135deg, #2193b0 0%, #6dd5ed 100%);
    color: white;
    border-radius: 8px;
    margin-bottom: 2rem;
}
"""

demo = gr.Blocks(css=custom_css, title="Python to Golang Converter", theme=gr.themes.Soft())

with demo:
    gr.HTML("""
    <div class="main-header">
        <h1>Python to Golang Code Converter</h1>
        <p>High-performance code conversion with runtime comparison</p>
    </div>
    """)
    
    hf_status = "✓ Configured" if HF_TOKEN else "✗ Not Found"
    hf_color = "green" if HF_TOKEN else "red"
    
    openai_status = "✓ Configured" if OPENAI_KEY else "✗ Not Found"
    openai_color = "green" if OPENAI_KEY else "red"
    
    gr.HTML(f"""
    <div style="padding: 1rem; background-color: #f8f9fa; border-radius: 8px; margin-bottom: 1rem;">
        <p style="margin: 0; color: {hf_color};"><strong>HF Token:</strong> {hf_status}</p>
        <p style="margin: 0.5rem 0 0 0; color: {openai_color};"><strong>OpenAI Key:</strong> {openai_status}</p>
        
    </div>
    """)
    
    with gr.Tabs():
        with gr.Tab("Convert Code"):
            with gr.Row():
                with gr.Column():
                    sample_selector = gr.Dropdown(
                        choices=list(SAMPLE_CODES.keys()),
                        label="Load Sample",
                        value="Hello World"
                    )
                    
                    python_editor = gr.Code(
                        label="Python Code",
                        value=SAMPLE_CODES["Hello World"],
                        language="python",
                        lines=15,
                        interactive=True
                    )
                    
                    model_selector = gr.Radio(
                        choices=list(ALL_MODELS.keys()),
                        label="Select Model",
                        value="GPT-4" if OPENAI_KEY else "CodeLlama-7B"
                    )
                    
                    with gr.Row():
                        convert_btn = gr.Button("Convert to Golang", variant="primary")
                        save_btn = gr.Button("Save to main.go", variant="secondary")
                
                with gr.Column():
                    golang_editor = gr.Textbox(
                        label="Generated Golang Code",
                        lines=15,
                        interactive=True,
                        show_copy_button=True
                    )
                    
                    save_status = gr.Textbox(
                        label="Save Status",
                        lines=1,
                        interactive=False
                    )
            
            with gr.Row():
                conversion_status = gr.Textbox(
                    label="Status",
                    lines=2,
                    interactive=False
                )
            
            with gr.Row():
                error_display = gr.Textbox(
                    label="Error Details",
                    lines=8,
                    interactive=False,
                    visible=True,
                    show_copy_button=True
                )
            
            sample_selector.change(
                fn=lambda x: load_sample(x),
                inputs=[sample_selector],
                outputs=[python_editor, error_display]
            )
            
            def convert_only(python_code, model_name):
                try:
                    if not python_code:
                        return "", "Error: No code", "Empty"
                    return convert_python_to_golang(python_code, model_name)
                except Exception as e:
                    return "", f"Error: {str(e)}", traceback.format_exc()
            
            convert_btn.click(
                fn=convert_only,
                inputs=[python_editor, model_selector],
                outputs=[golang_editor, conversion_status, error_display]
            )
            
            save_btn.click(
                fn=write_output,
                inputs=[golang_editor],
                outputs=[save_status]
            )
        
        with gr.Tab("Compare Models & Runtime"):
            gr.Markdown("### Compare All Models with Runtime Performance")
            
            with gr.Row():
                with gr.Column():
                    sample_selector2 = gr.Dropdown(
                        choices=list(SAMPLE_CODES.keys()),
                        label="Load Sample",
                        value="Pi Calculation"
                    )
                    
                    python_input2 = gr.Code(
                        label="Python Code",
                        value=SAMPLE_CODES["Pi Calculation"],
                        language="python",
                        lines=12,
                        interactive=True
                    )
                    
                    compare_btn = gr.Button("Compare All Models", variant="primary", size="lg")
            
            with gr.Row():
                comparison_summary = gr.Textbox(
                    label="Summary",
                    interactive=False,
                    lines=2
                )
            
            with gr.Row():
                comparison_table = gr.Dataframe(
                    label="Runtime Comparison Results",
                    interactive=False,
                    wrap=True
                )
            
            with gr.Row():
                compare_error = gr.Textbox(
                    label="Error Details",
                    lines=8,
                    interactive=False,
                    visible=True,
                    show_copy_button=True
                )
            
            sample_selector2.change(
                fn=lambda x: load_sample(x),
                inputs=[sample_selector2],
                outputs=[python_input2, compare_error]
            )
            
            compare_btn.click(
                fn=compare_all_models,
                inputs=[python_input2],
                outputs=[comparison_table, comparison_summary, compare_error]
            )
        

In [ ]:

#Launch
demo.launch()

